# **TUIA - Procesamiento de Imágenes y Visión por Computadora (IA52)**
# **Trabajo Práctico Final**
### **Ejercicio 4 - Evaluación**
<br>

### *Alumno: Miguel Mussi*
### *Año: 2024*

---------------------
## **Tabla de contenidos**
1.   [**Librerías**](#)
2.   [**Ajustes Iniciales**](#)
3.   [**Optimización de inferencia**](#)
4.   [**Inferencia sobre las imágenes y cálculo del envido**](#)
        1.   [*Inferencia con CPU*](#)
        2.   [*Inferencia con GPU - Sin TensorRT*](#)
        3.   [*Inferencia con GPU - Con TensorRT*](#)
        4.   [*Comparativa de Inferencias*](#)
5.   [**Guardamos las imagenes con las predicciones**](#)
6.   [**Escritura del archivo envido.json**](#)
7.   [**Exportar directorios**](#)

## 1. **Librerías**

In [18]:
!pip install ultralytics

In [19]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 37.4 MB/s eta 0:00:00


In [20]:
import os
import cv2
import time
import json
import torch
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. **Ajustes Iniciales**

In [21]:
# Nombre del alumno
student_name = 'miguel_mussi'

# Ruta al directorio principal
main_path = '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final'

# Ruta al archivo de pesos
model_path = os.path.join(main_path, 'model/weights/best.pt')
model_path_tensor = os.path.join(main_path, 'model/weights/best.onnx')

# Ruta al directorio que contiene las imagenes
imgs_dir = os.path.join(main_path, 'data/eval/images/val')

# Ruta al directorio de destino de las detecciones
base_dir = os.path.join(main_path, 'data/out')
dets_dir = os.path.join(base_dir, student_name)

# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)
os.makedirs(dets_dir)

In [11]:
# Seteo de tiempos de ejecución
execution_time = {'cpu': 0, 'gpu': 0, 'rt': 0}

## 3. **Optimización de inferencia**

In [12]:
# Cargamos el modelo
model = YOLO(model_path)

In [8]:
# Lo exportamos como un tensor para utilizar tensorRT con la GPU
model.export(format='onnx', imgsz=640, dynamic=True)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.30GHz)
Model summary (fused): 268 layers, 43,644,387 parameters, 0 gradients, 165.0 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 53, 8400) (83.6 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 15.1s, saved as '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights/best.onnx' (166.6 MB)

Export complete (23.8s)
Results saved to /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/C

'/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights/best.onnx'

### Comprobación de entorno gráfico

In [9]:
print(torch.cuda.is_available())

False


In [11]:
# print(f'Usando el dispositivo: {device}')

In [ ]:
print(f'Dispositivo del modelo: {next(model.parameters()).device}')

Dispositivo del modelo: cpu


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# device_id = 0  # El ID de dispositivo que deseas usar
# if device_id < torch.cuda.device_count():
#     device = torch.device(f'cuda:{device_id}')
# else:
#     raise AssertionError("Invalid device id")

In [12]:
num_gpus = torch.cuda.device_count()
print(f'Número de GPUs disponibles: {num_gpus}')

Número de GPUs disponibles: 0


## 4. **Inferencia sobre las imágenes y cálculo del envido**

### 4.1. *Inferencia con CPU*

#### V0

In [35]:
def calcular_envido(num1,num2):
    n1 = int(num1)
    n2 = int(num2)
    ret = n1 if n1<10 else 0
    ret += n2 if n2<10 else 0
    return ret + 20

# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            card_types[palo].append(num)

    envido = {}

    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3 or invalid_cards:
        print('La cantidad de cartas no permite calcular el envido.\n')

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo

        # calculamos el envido
        # Iteramos en el diccionario - Key = palos / Value: números
        for key, values in card_types.items():
            envido[key]=0
            # Dos cartas del mismo palo
            if len(values) == 1:
                envido[key] = int(num) if len(num)==1 else 0

            elif len(values) == 2:
                envido[key] = calcular_envido(values[0],values[1])

            # Tres cartas del mismo palo
            elif len(values) == 3:
                print('Mano con Flor.\n')
                envido[key] = calcular_envido(values[0],values[1])
                envido[key] = max(envido[key],calcular_envido(values[0],values[2]))
                envido[key] = max(envido[key],calcular_envido(values[1],values[2]))


    mejor_palo = 'N/A'
    mejor_punto = 0

    for palo, punto in envido.items():
        if punto > mejor_punto:
            mejor_punto=punto
            mejor_palo = palo

    # Almacenamos los resultados del envido en el diccionario para el JSON


    card_js_file[img_filename] = {
            'total_cards': len(filtered_cards),
            'cards': card_types,
            'points': mejor_punto,       # Puntos de envido
            'figure': mejor_palo          # Palo
        }


# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['cpu'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['cpu']:.2f} segundos")


image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 3181.6ms
La cantidad de cartas no permite calcular el envido.

image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 3497.8ms
image 3/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 2474.5ms
La cantidad de cartas no permite calcular el envido.

image 4/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 2454.9ms
image 5/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 2481.9ms
La cantida

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            card_types[palo].append(num)


    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3 or invalid_cards:
        print('La cantidad de cartas no permite calcular el envido.\n')
        envido = 0  # Aseguramos que envido tenga un valor
        palo = 'N/A'  # Aseguramos que palo tenga un valor

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo
        palo = 'N/A'

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
            envido = 0
            palo = 'N/A'

        # Si no hay cartas inválidas, calculamos el envido
        else:
            # Iteramos en el diccionario - Key = palos / Value: números
            for key, value in card_types.items():

                # Dos cartas del mismo palo
                if len(value) == 2:
                    palo = key
                    for num in value:
                            if num not in ['10', '11', '12']:
                                envido += int(num)

                # Tres cartas del mismo palo
                elif len(value) == 3:
                        print('Mano con Flor.\n')
                        palo = key
                        filtered_values = [x for x in value if x not in [10, 11, 12]]

                        # Filtramos los valores más altos exceptuando los 10, 11 y 12
                        if len(filtered_values) >= 2:
                            largest_values = sorted(filtered_values, reverse=True)[:2]
                            for num in largest_values:
                                envido += int(num)
                        elif len(filtered_values) == 1:
                            envido += int(filtered_values[0])
                        else:
                            envido = 20     # Valor trivial ya declarado
                else:
                    envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido >= 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,       # Puntos de envido
            'figure': palo          # Palo
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['cpu'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['cpu']:.2f} segundos")

#### v1

In [20]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Iteramos sobre los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
        envido = 0
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,       # Puntos de envido
            'figure': type          # Palo
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time_cpu = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time_cpu:.2f} segundos')


image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 2500.0ms
image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 2379.1ms
image 3/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 2637.6ms
image 4/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 3789.7ms
image 5/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 2980.7ms
La cantidad de cartas no permite calcular el envido

image 6/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión

### 4.2. *Inferencia con GPU - Sin TensorRT*

#### V0

In [27]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
# device = torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas al dispositivo GPU
    cards = result.to(device)
    # cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            card_types[palo].append(num)


    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3:
        print('La cantidad de cartas no permite calcular el envido.\n')
        envido = 0  # Aseguramos que envido tenga un valor
        palo = 'N/A'  # Aseguramos que palo tenga un valor

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo
        palo = 'N/A'

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
            envido = 0
            palo = 'N/A'

        # Si no hay cartas inválidas, calculamos el envido
        else:
            # Iteramos en el diccionario - Key = palos / Value: números
            for key, value in card_types.items():

                # Dos cartas del mismo palo
                if len(value) == 2:
                    palo = key
                    for num in value:
                            if num not in ['10', '11', '12']:
                                envido += int(num)

                # Tres cartas del mismo palo
                elif len(value) == 3:
                        print('Mano con Flor.\n')
                        palo = key
                        filtered_values = [x for x in value if x not in [10, 11, 12]]

                        # Filtramos los valores más altos exceptuando los 10, 11 y 12
                        if len(filtered_values) >= 2:
                            largest_values = sorted(filtered_values, reverse=True)[:2]
                            for num in largest_values:
                                envido += int(num)
                        elif len(filtered_values) == 1:
                            envido += int(filtered_values[0])
                        else:
                            envido = 20     # Valor trivial ya declarado
                else:
                    envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido >= 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,       # Puntos de envido
            'figure': palo          # Palo
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['gpu'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['gpu']:.2f} segundos")


image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 3549.9ms
La cantidad de cartas no permite calcular el envido.

image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 2431.2ms
image 3/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 2665.8ms
La cantidad de cartas no permite calcular el envido.

image 4/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 2463.3ms
image 5/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 3362.2ms
La cantida

#### V1

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cuda:0')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Procesamos los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,
            'figure': type
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time_gpu = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time_gpu:.2f} segundos')

### 4.3. *Inferencia con GPU - Con TensorRT*

#### V0

In [23]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
model = YOLO(model_path_tensor, task='detect')

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas al dispositivo GPU
    cards = result.to(device)
    # cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            card_types[palo].append(num)


    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3:
        print('La cantidad de cartas no permite calcular el envido.\n')
        envido = 0  # Aseguramos que envido tenga un valor
        palo = 'N/A'  # Aseguramos que palo tenga un valor

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo
        palo = 'N/A'

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
            envido = 0
            palo = 'N/A'

        # Si no hay cartas inválidas, calculamos el envido
        else:
            # Iteramos en el diccionario - Key = palos / Value: números
            for key, value in card_types.items():

                # Dos cartas del mismo palo
                if len(value) == 2:
                    palo = key
                    for num in value:
                            if num not in ['10', '11', '12']:
                                envido += int(num)

                # Tres cartas del mismo palo
                elif len(value) == 3:
                        print('Mano con Flor.\n')
                        palo = key
                        filtered_values = [x for x in value if x not in [10, 11, 12]]

                        # Filtramos los valores más altos exceptuando los 10, 11 y 12
                        if len(filtered_values) >= 2:
                            largest_values = sorted(filtered_values, reverse=True)[:2]
                            for num in largest_values:
                                envido += int(num)
                        elif len(filtered_values) == 1:
                            envido += int(filtered_values[0])
                        else:
                            envido = 20     # Valor trivial ya declarado
                else:
                    envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido >= 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,       # Puntos de envido
            'figure': palo          # Palo
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['rt'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['rt']:.2f} segundos")

Loading /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/model/weights/best.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 150.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.9s, installed 1 package: ['onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x640 1 4O, 1 11C, 2809.3ms
La cantidad de cartas no permite calcular el envido.

image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x6

#### V1

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
model = YOLO(model_path_tensor, task='detect')

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Procesamos los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,
            'figure': type
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time_rt = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time_rt:.2f} segundos')

### 4.4. *Comparativa de Inferencias*

In [28]:
# Comparación de los tiempos de ejecución
print("Comparación de Tiempos de Ejecución:")
print(f"Tiempo de ejecución (CPU): {execution_time['cpu']:.2f} segundos")
print(f"Tiempo de ejecución (GPU): {execution_time['gpu']:.2f} segundos")
print(f"Tiempo de ejecución (TensorRT): {execution_time['rt']:.2f} segundos")

Comparación de Tiempos de Ejecución:
Tiempo de ejecución (CPU): 113.86 segundos
Tiempo de ejecución (GPU): 100.84 segundos
Tiempo de ejecución (TensorRT): 135.22 segundos


## 5. **Guardamos las imagenes con las predicciones**

In [ ]:
# Cargamos el modelo
# model = YOLO(model_path)              # Con CPU
model = YOLO(model_path).to(device)     # Con GPU (sin TensorRT)
# model = YOLO(model_path_tensor)       # Con GPU (con TensorRT)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Directorio donde van a ir las deteciones
detection_dir = f'{base_dir}/detections/'

# Creamos el directorio si no existe
os.makedirs(detection_dir, exist_ok=True)

# Iteramos sobre los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Guardamos la detección
    result.save(filename=f'{detection_dir}/{img_filename}')


image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 1805.6ms
image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 1304.6ms
image 3/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 1376.4ms
image 4/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1357.8ms
image 5/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 1363.4ms
image 6/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20

## 6. **Escritura del archivo envido.json**

In [36]:
with open(os.path.join(dets_dir, 'envido.json'), 'w') as jf:
    json.dump(card_js_file, jf, indent=4)

## 7. **Exportar directorios**

In [28]:
# Especifica la carpeta a comprimir y el nombre del archivo zip
folder_path = '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections'

zip_name = '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip'

# Crear el archivo zip
shutil.make_archive(zip_name.replace('.zip', ''), 'zip', folder_path)

print(f"Archivo zip creado: {zip_name}")

Archivo zip creado: /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip


In [27]:
# Eliminar un archivo
os.remove('/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip')
print("Eliminación completa!")

Eliminación completa!


In [ ]:
# Eliminar un directorio y todo su contenido
shutil.rmtree('/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections')
print("Eliminación completa!")

Eliminación completa!
